<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

Инвестор может получить доход двумя способами:
* положить деньги в банк по 6% в год
* купить акции, для которых прогнозируется рост в цене на 13% в год

Инвестор хочет снизить риски, поэтому инвестирует в акции не более 60% доступных средств и кладет на вклад не менее 25%.

Определим, как инвестору распределить свой капитал, чтобы максимизировать доход от вложений.

Целевая функция:

x1 * 0.06 + x2 * 0.13 -> max

Ограничения:

x1 + 0 * x2 => 0;

0 * x1 + x2 => 0;

x1 + 0 * x2 <= 1;

0 * x1 + x2 <= 1;

x1 + 0 * x2 >= 0.25;

0 * x1 + x2 <= 0.6.

In [1]:
import sympy as sp
import pandas as pd
import numpy as np 

In [2]:
with open('demo.txt', 'r') as file:
    input_data = file.read()
    print(input_data)

x0 * 0.06 + x1 * 0.13, max
x0 * 1 + x1 * 0, <=, 1
x0 * 0 + x1 * 1, <=, 1
x0 * 1 + x1 * 0, >=, 0.25
x0 * 0 + x1 * 1, <=, 0.6
x0 * 1 + x1 * 1, <=, 1


In [3]:
# Парсинг входных данных
def parse_formula(formula_str):
    x0, x1 = sp.symbols('x0 x1')
    func = sp.sympify(formula_str)
    coeffs = func.as_coefficients_dict()
    res = []
    for x in [x0,x1]:
        res.append(coeffs[x])
    return res

# Приведение к каноническому виду
def to_canonical(f, cons):
    if f[1] == 'min':
        f = f[0].append(0)
    else:
        f = [(-1) * i for i in f[0]]
        f.append(0)
        
    constr = []
    for c in cons:
        if c[1] == '<=':
            c[0].append(1)
            co = []
            co.append(c[0])
            co.append('=')
            co.append(c[2])
            constr.append(co)
        elif c[1] == '>=':
            co = []
            c[0].append(-1)
            co.append([(-1) * i for i in c[0]])
            co.append('=')
            co.append(c[2] * (-1))
            constr.append(co)
    return f, constr

# Создание матрицы
def create_matrix(res):
    m = []
    f = ['base', 'free']
    f.extend([f'x{i}' for i in range(len(res[0])-1)])
    m.append(f)
    for i in range(len(res[1])):
        row = [f'y{i}']
        row.append(res[1][i][-1])
        row.extend(res[1][i][0])
        row = row[:-1]
        row.append('')
        m.append(row)
    
    row = ['F']
    row.append(res[0][-1])
    row.extend(res[0][:-1])
    row.append('')
    m.append(row)
    return m

In [4]:
def prepare_data(input_data):
    # Преобразование целевой функции
    obj_func_str, obj_type = input_data.splitlines()[0].split(', ')
    obj_func_coeffs = parse_formula(obj_func_str)
    obj_func = [obj_func_coeffs, obj_type]
    
    # Преобразование ограничений
    constraints = []
    for constraint_str in input_data.splitlines()[1:]:
        constraint = constraint_str.split(', ')
        sign, value = constraint[1], float(constraint[2])
        coeffs = parse_formula(constraint[0])
        constraints.append([coeffs, sign, value])
        
    # Приведение к каноническому виду
    canon_matrix = to_canonical(obj_func, constraints)
    # Преобразование в матрицу
    matrix = create_matrix(canon_matrix)
    return matrix

In [5]:
matrix = prepare_data(input_data)

In [6]:
def simplex_method(matrix):
    while np.min(matrix[-1][2:-1]) < 0:
        ind_stol = np.argmin(matrix[-1][2:-1]) + 2
        ved_stol = [i[ind_stol] for i in matrix[1:]]
        svob_ch = [i[1] for i in matrix[1:]]
        ratios = [np.inf if y == 0 else np.inf if (x > 0) & (y < 0) else x/y for x,y in list(zip(svob_ch[:-1], ved_stol[:-1]))]
        ind_str = np.argmin(ratios)
        ved_str = matrix[ind_str+1][1:-1]
        
        ved_el = matrix[ind_str+1][ind_stol]
        # ищем вспомогательные коэфы
        vsp_coef = [i*(-1) for i in ved_stol]
        vsp_coef[ind_str] = ''
        # ищем ведущую строку
        ved_str = [i/ved_el for i in ved_str]
        ved_str[ind_stol-1] = 1/ved_el
        
        # ищем остальные
        new_matr = []
        for i, row in enumerate(matrix[1:]):
            if i != ind_str:
                new_row = [row[0]]
                new_row.extend([s * vsp_coef[i] + row[n+1] if n!=ind_stol-1 else s * vsp_coef[i] for n, s in enumerate(ved_str)])
                new_row.append(vsp_coef[i])
                new_matr.append(new_row)
            else:
                f_ved_str = ved_str.copy()
                f_ved_str.insert(0, matrix[0][ind_stol])
                f_ved_str.append('')
                new_matr.append(f_ved_str)
        matrix[0][ind_stol] = matrix[ind_str+1][0]
        new_matr.insert(0, matrix[0])
        matrix = new_matr.copy()
        display(pd.DataFrame(new_matr[1:], columns=['variable']+new_matr[0]))
    final_table = pd.DataFrame(new_matr[1:], columns=['variable']+new_matr[0])
    result = {}
    for var in final_table.variable:
        if var[0] == 'x':
            result[var] = final_table[final_table["variable"] == var]["base"].values[0]
    return result

In [7]:
result = simplex_method(matrix)

print('Результаты')
for k, v in result.items():
    print(f'{k} = {round(v, 2)}')

,variable,base,free,x0,y3
0,y0,1.00000000000000,1,0,0
1,y1,0.400000000000000,0,-1,-1
2,y2,-0.250000000000000,-1,0,0
3,x1,0.600000000000000,0,1,
4,y4,0.400000000000000,1,-1,-1
5,F,0.0780000000000000,-0.0600000000000000,0.130000000000000,0.130000000000000


,variable,base,free,y2,y3
0,y0,0.750000000000000,1,0,-1
1,y1,0.400000000000000,0,-1,0
2,x0,0.250000000000000,-1,0,
3,x1,0.600000000000000,0,1,0
4,y4,0.150000000000000,1,-1,-1
5,F,0.0930000000000000,-0.0600000000000000,0.130000000000000,0.0600000000000000


,variable,base,free,y4,y3
0,y0,0.600000000000000,-1,1,-1
1,y1,0.400000000000000,0,-1,0
2,x0,0.400000000000000,1,-1,1
3,x1,0.600000000000000,0,1,0
4,y2,0.150000000000000,1,-1,
5,F,0.102000000000000,0.0600000000000000,0.0700000000000000,0.0600000000000000


Результаты
x0 = 0.40
x1 = 0.60
